In [ ]:
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install tqdm
!{sys.executable} -m pip install pyxlsb
!{sys.executable} -m pip install pandas_market_calendars

In [ ]:
import pandas as pd
import warnings
import pandas_market_calendars as mcal
from pyxlsb import open_workbook
from tqdm import tqdm
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [ ]:
file_path = "../data/"

df_purchase = pd.read_excel(file_path + "base2023_compra.xlsb", engine="pyxlsb")
df_purchase_2 = pd.read_excel(file_path + "base2023_compra_2.xlsb", engine="pyxlsb")

In [ ]:
# change the date columns to pandas datetime
df_purchase_datetime = df_purchase.copy(deep=True)
df_purchase_2_datetime = df_purchase_2.copy(deep=True)

df_purchase_datetime['Dt. Operação'] = pd.to_datetime(df_purchase_datetime['Dt. Operação'], unit='D', origin='1899-12-30')
df_purchase_2_datetime['Dt. Operação'] = pd.to_datetime(df_purchase_2_datetime['Dt. Operação'], format="%d/%m/%Y")

df_purchase_2_datetime.rename(columns={"Vencimento": "Dt. Liquidação"}, inplace=True)

df_purchase_datetime['Dt. Liquidação'] = pd.to_datetime(df_purchase_datetime['Dt. Liquidação'], unit='D', origin='1899-12-30')
df_purchase_2_datetime['Dt. Liquidação'] = pd.to_datetime(df_purchase_2_datetime['Dt. Liquidação'], errors='coerce', format="%d/%m/%Y")

df_purchases = pd.concat([df_purchase_datetime, df_purchase_2_datetime], ignore_index=True)
df_purchases = df_purchases.sort_values(by=["Cód. Cliente", "Dt. Operação", "Cód. Título", "Cód. Corretora"]).reset_index()
pd.set_option('display.max_columns', None)

In [ ]:
df_sales = pd.read_excel(file_path + "base2023_venda.xlsb", engine="pyxlsb")

df_sales_datetime = df_sales.copy(deep=True)
df_sales_datetime['Dt. Operação'] = pd.to_datetime(df_sales_datetime['Dt. Operação'], unit='D', origin='1899-12-30')
df_sales_datetime['Vencimento'] = pd.to_datetime(df_sales_datetime['Vencimento'], unit='D', origin='1899-12-30')
df_sales_datetime = df_sales_datetime.sort_values(by=["Cód. Cliente", "Dt. Operação", "Cód. Título", "Cód. Corretora"]).reset_index()

df_filtered_purchases = df_purchases.copy(deep=True)
df_filtered_sales = df_sales_datetime.copy(deep=True)

# removing unused columns
df_filtered_purchases = df_filtered_purchases.drop(["index", "Tipo Operação", "Valor Líquido", "Dt. Liquidação", "Tipo"], axis=1)
df_filtered_sales = df_filtered_sales.drop(["index", "Tipo Operação", "Valor Líquido", "Tipo"], axis=1)

In [ ]:
brazil_calendar = mcal.get_calendar('BMF')

def calculate_du(x):
  """Calculates the number of business days between the sale operation date and the expiration date

  Args:
      x (dataframe): dataframe with the sales operations

  Returns:
      int: number of business days
  """
  business_days = brazil_calendar.valid_days(x["Dt. Operação"], x["Vencimento"])
  du = len(business_days) - 1
  return du

# apply the calculate_du function to each row of the dataframe
df_filtered_sales["du"] = df_filtered_sales.apply(calculate_du, axis=1)

df_filtered_sales_du = df_filtered_sales.copy(deep=True)

df_filtered_sales_du = df_filtered_sales[(df_filtered_sales["du"] > 0)].sort_values(by=['Cód. Cliente', 'Dt. Operação', 'Cód. Título', 'Cód. Corretora'])
display(df_filtered_sales_du.head())

,Chave,Cód. Cliente,Dt. Operação,Cód. Título,Cód. Corretora,Quantidade,Preço,Vencimento,DI,du
0,4721A,Cliente 1,2023-01-05,XPBR31,XPIN,4470,76.450544,2023-03-21,0.136781,51
1,19007A,Cliente 1,2023-01-05,XPBR31,XPIN,35530,76.440559,2023-03-21,0.136781,51
2,20955A,Cliente 1,2023-01-05,XPBR31,XPIN,5440,76.069360,2023-03-06,0.136704,40
3,30031A,Cliente 1,2023-01-05,XPBR31,XPIN,14560,76.079336,2023-03-06,0.136704,40
4,18899A,Cliente 1,2023-01-19,XPBR31,XPIN,41230,86.807599,2023-02-22,0.136585,22


In [ ]:
# Calculate the index to split on
split_index = len(df_filtered_sales_du) // 2

# Split into two DataFrames
df_part1 = df_filtered_sales_du.iloc[:split_index + 3]
df_part2 = df_filtered_sales_du.iloc[split_index + 3:]

In [ ]:
# create a list of dictionaries with the sales and their respective purchases that can be combined. This will be used to train the agent
filtered_dfs = []

for index, row in tqdm(df_part1.iterrows()):
  available_purchases = df_filtered_purchases[
    (df_filtered_purchases['Cód. Cliente'] == row['Cód. Cliente']) &
    (df_filtered_purchases['Dt. Operação'] == row['Dt. Operação']) &
    (df_filtered_purchases['Cód. Corretora'] == row['Cód. Corretora']) &
    (df_filtered_purchases['Cód. Título'] == row['Cód. Título'])
  ]

  if not available_purchases.empty:
    filtered_dfs.append({
        'sale': row.copy(deep=True),
        'purchase': available_purchases.copy(deep=True)
    })

In [ ]:
# save the dictionary in .txt file, so it can be used without running all the code again
import pickle
file = open('../data/dataframes_train.txt', 'wb')
pickle.dump(filtered_dfs, file)
file.close()

In [ ]:
# create a list of dictionaries with the sales and their respective purchases that can be combined. This will be used to test the agent
test_filtered_dfs = []

for index, row in tqdm(df_part2.iterrows()):
  available_purchases = df_filtered_purchases[
    (df_filtered_purchases['Cód. Cliente'] == row['Cód. Cliente']) &
    (df_filtered_purchases['Dt. Operação'] == row['Dt. Operação']) &
    (df_filtered_purchases['Cód. Corretora'] == row['Cód. Corretora']) &
    (df_filtered_purchases['Cód. Título'] == row['Cód. Título'])
  ]

  if not available_purchases.empty:
    test_filtered_dfs.append({
        'sale': row.copy(deep=True),
        'purchase': available_purchases.copy(deep=True)
    })

In [ ]:
# save the dictionary in .txt file, so it can be used without running all the code again
import pickle
file = open('../data/dataframes_test.txt', 'wb')
pickle.dump(test_filtered_dfs, file)
file.close()